In [1]:
import numpy as np
import pandas as pd
import sqlite3 as db
import random
from datetime import date, timedelta, datetime
from faker import Faker
import os
from itertools import product

## Create A Database for Healthcare Data and A Database for Patient Data

In [2]:
hb = [['Capital Region', 'CPR', 20000], 
             ['Forest Heights', 'FRH', 11000], 
             ['Middlepark', 'MPK', 8000],
             ['The Dale', 'DLE', 1500]]

hb_df = pd.DataFrame(hb, columns=['Health Board', 'Health Board Code', 'Population'])

hospitals = [['University Hospital Capital','UHC123','Acute','CPR'],
             ['Downtown Hospital','DTH111','Non Acute','CPR'],
             ['Southside General','SSG001','Acute','CPR'],
             ['Fir Tree General','FTG221','Non Acute','FRH'],
             ['Oaken Ridge Infirmary','ORI010','Acute','FRH'],
             ['New Hope Hospital','NHH131','Acute','MPK'],
             ['Bright Dawn Hospital','BDH101','Acute','MPK'],
             ['The Valley Infirmary','TVI854','Acute','DLE']]

hospitals_df = pd.DataFrame(hospitals, columns=['Hospital Name', 
                                                'Hospital Code', 
                                                'Acute Status', 
                                                'Health Board Code'])

wards = [['ICU A','UHC:ICUA','UHC123'],
        ['ICU B','UHC:ICUB','UHC123'],
        ['A&E',	'UHC:AE','UHC123'],
        ['Ward 1','UHC:1','UHC123'],
        ['Ward 2','UHC:2','UHC123'],
        ['Maternity','UHC:MAT','UHC123'],
        ['Palliative','DTH:PAL','DTH111'],
        ['Ward 1','DTH:1','DTH111'],
        ['Ward 2','DTH:2','DTH111'],
        ['Paediatrics','SSG:PAE','SSG001'],
        ['Oncology','SSG:ONC','SSG001'],
        ['A&E','SSG:AE','SSG001'],
        ['Geriatrics','FTG:GER','FTG221'],
        ['Ward 1','FTG:1','FTG221'],
        ['Psychiatic','FTG:PSY','FTG221'],
        ['Internal Medicine','ORI:INTM','ORI010'],
        ['General Medicine','ORI:GENM','ORI010'],
        ['Ward 1','ORI:1','ORI010'],
        ['Ward 1','NHHL1','NHH131'],
        ['Respiratory','NHH:RES','NHH131'],
        ['ICU A','BGH:ICUA','BDH101'],
        ['ICU B','BGH:ICUB','BDH101'],
        ['Ward 1','BGH:1','BDH101'],
        ['A&E','TVI:AE','TVI854']]

wards_df = pd.DataFrame(wards, columns=['Ward Name', 
                                                'Ward Code',
                                                'Hospital Code'])

# Set seed for reproducibility
random.seed(123)

# Initialize faker
fake = Faker()

# Define the number of entries for the dataset
num_entries = 5000

# Generate fake data for each column
patient_id = [random.randint(100000, 999999) for _ in range(num_entries)]
age = [random.randint(18, 90) for _ in range(num_entries)]
sex = [random.choice(['Male', 'Female']) for _ in range(num_entries)]
unit_admission_date = [fake.date_between_dates(date(2019, 1, 1), date.today()) for _ in range(num_entries)]
unit_discharge_date = [admission + timedelta(days=random.randint(0, 30)) for admission in unit_admission_date]
specimen_date = [fake.date_between_dates(admission, discharge) if random.random() < 0.3 else fake.date_between(start_date=date(2019, 1, 1), end_date=date.today()) for admission, discharge in zip(unit_admission_date, unit_discharge_date)]
organism = [random.choice(["Staphylococcus aureus", "Escherichia coli", "Pseudomonas aeruginosa", "Enterococcus faecalis", "Klebsiella pneumoniae"]) for _ in range(num_entries)]
ward = [random.choice(wards_df['Ward Code'].unique()) for _ in range(num_entries)]

# Create the patient data dictionary
patient_data = {
    "Patient ID": patient_id,
    "Age": age,
    "Sex": sex,
    "Unit Admission Date": unit_admission_date,
    "Unit Discharge Date": unit_discharge_date,
    "Specimen Date": specimen_date,
    "Organism": organism,
    "Ward Code": ward
}
patient_data_df = pd.DataFrame.from_dict(patient_data)

con = db.connect(os.path.realpath('HealthCareData.db'))
hb_df.to_sql('hb', con, if_exists='replace')
hospitals_df.to_sql('hospitals', con, if_exists='replace')
wards_df.to_sql('wards', con, if_exists='replace')
con.close()
con = db.connect(os.path.realpath('PatientData.db'))
patient_data_df.to_sql('patients', con, if_exists='replace')
con.close()


## Read in data from databases

In [3]:
# Connect to healthcare database
conn = db.connect('HealthCareData.db')

# Create cursor object
cursor = conn.cursor()

# Query for LEFT JOIN
sql_query = '''SELECT 
w.[Ward Name], w.[Ward Code], w.[Hospital Code],
h.[Hospital Name], h.[Acute Status], h.[Health Board Code],
hb.[Health Board], hb.[Population]
FROM wards w
LEFT OUTER JOIN hospitals h
ON w.[Hospital Code] = h.[Hospital Code]
LEFT OUTER JOIN hb 
ON h.[Health Board Code] = hb.[Health Board Code];'''

# Executing the query
cursor.execute(sql_query)

# Get column names
headers = [i[0] for i in cursor.description]

# Fetch rows from the result table and assign column names
health_data = pd.DataFrame(cursor.fetchall())
health_data.columns = headers
conn.close()
health_data


,Ward Name,Ward Code,Hospital Code,Hospital Name,Acute Status,Health Board Code,Health Board,Population
0,ICU A,UHC:ICUA,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
1,ICU B,UHC:ICUB,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
2,A&E,UHC:AE,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
3,Ward 1,UHC:1,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
4,Ward 2,UHC:2,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
5,Maternity,UHC:MAT,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000
6,Palliative,DTH:PAL,DTH111,Downtown Hospital,Non Acute,CPR,Capital Region,20000
7,Ward 1,DTH:1,DTH111,Downtown Hospital,Non Acute,CPR,Capital Region,20000
8,Ward 2,DTH:2,DTH111,Downtown Hospital,Non Acute,CPR,Capital Region,20000
9,Paediatrics,SSG:PAE,SSG001,Southside General,Acute,CPR,Capital Region,20000


In [4]:
# Connect to patient database
conn = db.connect('PatientData.db')

# Create cursor object
cursor = conn.cursor()

# Query for LEFT JOIN
sql_query = '''SELECT p.[Patient ID], p.[Age], p.[Sex], p.[Unit Admission Date],
p.[Unit Discharge Date], p.[Specimen Date], p.[Organism], p.[Ward Code]
FROM patients p;'''

# Executing the query
cursor.execute(sql_query)

# Get column names
headers = [i[0] for i in cursor.description]

# Fetch rows from the result table and assign column names
patient_data = pd.DataFrame(cursor.fetchall())
patient_data.columns = headers
conn.close()
patient_data.head(5)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code
0,154907,49,Male,2019-10-05,2019-11-04,2021-05-31,Staphylococcus aureus,DTH:PAL
1,380679,28,Female,2019-05-20,2019-06-10,2022-07-09,Enterococcus faecalis,FTG:PSY
2,191421,37,Female,2019-06-30,2019-07-24,2023-09-04,Enterococcus faecalis,SSG:ONC
3,906309,51,Male,2021-08-17,2021-09-12,2021-08-19,Escherichia coli,UHC:ICUA
4,527023,84,Male,2020-12-12,2021-01-09,2019-08-03,Klebsiella pneumoniae,DTH:1


## Identify patients whose positive sample occured on day 3 or later of a stay in healthcare (to account for incubation time of organism). Day of admission is taken as day 1 of healthcare stay

In [6]:
patient_data ["Specimen Date"]= pd.to_datetime(patient_data['Specimen Date'])
patient_data["Unit Admission Date"] = pd.to_datetime(patient_data["Unit Admission Date"])
patient_data["Unit Discharge Date"] = pd.to_datetime(patient_data["Unit Discharge Date"])
patient_data['HAI'] = np.where((patient_data['Specimen Date'] >= patient_data['Unit Admission Date']+timedelta(days=2)) & (patient_data['Specimen Date'] <= patient_data['Unit Discharge Date']),1,0)
patient_data['Days To Infection'] = (patient_data['Specimen Date'] - patient_data['Unit Admission Date']).dt.days
patient_data.head(10)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection
0,154907,49,Male,2019-10-05,2019-11-04,2021-05-31,Staphylococcus aureus,DTH:PAL,0,604
1,380679,28,Female,2019-05-20,2019-06-10,2022-07-09,Enterococcus faecalis,FTG:PSY,0,1146
2,191421,37,Female,2019-06-30,2019-07-24,2023-09-04,Enterococcus faecalis,SSG:ONC,0,1527
3,906309,51,Male,2021-08-17,2021-09-12,2021-08-19,Escherichia coli,UHC:ICUA,1,2
4,527023,84,Male,2020-12-12,2021-01-09,2019-08-03,Klebsiella pneumoniae,DTH:1,0,-497
5,379501,90,Female,2023-06-16,2023-06-25,2023-06-19,Enterococcus faecalis,SSG:PAE,1,3
6,212931,50,Male,2022-07-15,2022-07-28,2022-07-22,Staphylococcus aureus,ORI:INTM,1,7
7,979234,28,Female,2021-05-01,2021-05-03,2023-03-27,Pseudomonas aeruginosa,BGH:ICUB,0,695
8,140007,19,Female,2021-02-28,2021-03-16,2019-02-28,Enterococcus faecalis,SSG:PAE,0,-731
9,497540,71,Female,2019-07-03,2019-07-24,2020-10-13,Escherichia coli,SSG:ONC,0,468


### Create Age Groups and Length Of Stay for patient data. 

In [7]:
# List of conditions
conditions = [
      (patient_data["Age"] >= 0) & (patient_data["Age"] <= 16)
    , (patient_data["Age"] > 16) & (patient_data["Age"] <= 30)
    , (patient_data["Age"] > 30) & (patient_data["Age"] <= 50)
    , (patient_data["Age"] > 50) & (patient_data["Age"] <= 65)
    , (patient_data["Age"] > 65)
]
# List of values to return
choices  = [
      "0-16"
    , "17-30"
    , "31-50"
    , "51-65"
    , "65+"
]
# create a new column in the DF based on the conditions
patient_data["Age Group"] = np.select(conditions, choices, "ERROR")
patient_data['Length Of Stay'] = (patient_data['Unit Discharge Date'] - patient_data['Unit Admission Date']).dt.days+1
patient_data.head(5)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection,Age Group,Length Of Stay
0,154907,49,Male,2019-10-05,2019-11-04,2021-05-31,Staphylococcus aureus,DTH:PAL,0,604,31-50,31
1,380679,28,Female,2019-05-20,2019-06-10,2022-07-09,Enterococcus faecalis,FTG:PSY,0,1146,17-30,22
2,191421,37,Female,2019-06-30,2019-07-24,2023-09-04,Enterococcus faecalis,SSG:ONC,0,1527,31-50,25
3,906309,51,Male,2021-08-17,2021-09-12,2021-08-19,Escherichia coli,UHC:ICUA,1,2,51-65,27
4,527023,84,Male,2020-12-12,2021-01-09,2019-08-03,Klebsiella pneumoniae,DTH:1,0,-497,65+,29


### Merge the datasets together describing the healthcare setting and patient information based on Ward Code

In [8]:
df = pd.merge(patient_data,health_data,on='Ward Code',how='left')
df['Month_Year'] = df['Specimen Date'].dt.to_period('M')
df.head(5)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection,Age Group,Length Of Stay,Ward Name,Hospital Code,Hospital Name,Acute Status,Health Board Code,Health Board,Population,Month_Year
0,154907,49,Male,2019-10-05,2019-11-04,2021-05-31,Staphylococcus aureus,DTH:PAL,0,604,31-50,31,Palliative,DTH111,Downtown Hospital,Non Acute,CPR,Capital Region,20000,2021-05
1,380679,28,Female,2019-05-20,2019-06-10,2022-07-09,Enterococcus faecalis,FTG:PSY,0,1146,17-30,22,Psychiatic,FTG221,Fir Tree General,Non Acute,FRH,Forest Heights,11000,2022-07
2,191421,37,Female,2019-06-30,2019-07-24,2023-09-04,Enterococcus faecalis,SSG:ONC,0,1527,31-50,25,Oncology,SSG001,Southside General,Acute,CPR,Capital Region,20000,2023-09
3,906309,51,Male,2021-08-17,2021-09-12,2021-08-19,Escherichia coli,UHC:ICUA,1,2,51-65,27,ICU A,UHC123,University Hospital Capital,Acute,CPR,Capital Region,20000,2021-08
4,527023,84,Male,2020-12-12,2021-01-09,2019-08-03,Klebsiella pneumoniae,DTH:1,0,-497,65+,29,Ward 1,DTH111,Downtown Hospital,Non Acute,CPR,Capital Region,20000,2019-08


### For each patient, find the day of admission, day of dischargeand fill in all the dates in between.

In [14]:
# Function to get all dates between admission and discharge date
def get_dates(row):
    return pd.date_range(start=row['Unit Admission Date'], end=row['Unit Discharge Date']).tolist()

# Create a list to store the results
all_dates = []

# Iterate through each row of the original DataFrame and append dates to the list
for _, row in df.iterrows():
    dates_between = get_dates(row)
    all_dates.extend(dates_between)

# Create a new DataFrame from the list of dates
bed_days_df = pd.DataFrame({'All_Dates': all_dates})

bed_days_df.head(40)

,All_Dates
0,2019-10-05
1,2019-10-06
2,2019-10-07
3,2019-10-08
4,2019-10-09
5,2019-10-10
6,2019-10-11
7,2019-10-12
8,2019-10-13
9,2019-10-14


### Create a new dataframe that we can use to group demographic data and count how many bed days were occupied by each demographic each month. First, multrepeat each patient's demographic data the same number as days spent in healthcare (ie a patient in a ward for 3 days will have 3 rows of data)

In [15]:
# Function to repeat IDs and Ages based on the value in 'Length of Stay' column
def repeat_ids_and_ages(row):
    return pd.DataFrame({'Sex': [row['Sex']] * row['Length Of Stay'], 
                         'Age Group': [row['Age Group']] * row['Length Of Stay'], 
                         'Ward Code': [row['Ward Code']] * row['Length Of Stay'],
                         'Health Board': [row['Health Board']] * row['Length Of Stay']})

# Create a list to store the repeated IDs and Ages
all_data = []

# Iterate through each row of the original DataFrame and append repeated IDs and Ages to the list
for _, row in df.iterrows():
    repeated_data = repeat_ids_and_ages(row)
    all_data.append(repeated_data)

# Concatenate all the repeated data DataFrames into a single DataFrame
demographics_df = pd.concat(all_data, ignore_index=True)

demographics_df.head(10)


,Sex,Age Group,Ward Code,Health Board
0,Male,31-50,DTH:PAL,Capital Region
1,Male,31-50,DTH:PAL,Capital Region
2,Male,31-50,DTH:PAL,Capital Region
3,Male,31-50,DTH:PAL,Capital Region
4,Male,31-50,DTH:PAL,Capital Region
5,Male,31-50,DTH:PAL,Capital Region
6,Male,31-50,DTH:PAL,Capital Region
7,Male,31-50,DTH:PAL,Capital Region
8,Male,31-50,DTH:PAL,Capital Region
9,Male,31-50,DTH:PAL,Capital Region


### Attach the column of each date that a patient was in healthcare

In [16]:
bed_days_df = pd.concat([bed_days_df.reset_index(drop=True), demographics_df], axis=1)
bed_days_df.head(5)

,All_Dates,Sex,Age Group,Ward Code,Health Board
0,2019-10-05,Male,31-50,DTH:PAL,Capital Region
1,2019-10-06,Male,31-50,DTH:PAL,Capital Region
2,2019-10-07,Male,31-50,DTH:PAL,Capital Region
3,2019-10-08,Male,31-50,DTH:PAL,Capital Region
4,2019-10-09,Male,31-50,DTH:PAL,Capital Region


### Replace the full dates with a 'Month_Year' combo. This is the date stratification that will be used in the resuting dashboard.

In [17]:
bed_days_df['Month_Year'] = bed_days_df['All_Dates'].dt.to_period('M')
bed_days_df = bed_days_df.drop('All_Dates', axis=1)
bed_days_df = bed_days_df[['Month_Year', 'Sex', 'Age Group', 'Ward Code', 'Health Board']]
bed_days_df.head(40)

,Month_Year,Sex,Age Group,Ward Code,Health Board
0,2019-10,Male,31-50,DTH:PAL,Capital Region
1,2019-10,Male,31-50,DTH:PAL,Capital Region
2,2019-10,Male,31-50,DTH:PAL,Capital Region
3,2019-10,Male,31-50,DTH:PAL,Capital Region
4,2019-10,Male,31-50,DTH:PAL,Capital Region
5,2019-10,Male,31-50,DTH:PAL,Capital Region
6,2019-10,Male,31-50,DTH:PAL,Capital Region
7,2019-10,Male,31-50,DTH:PAL,Capital Region
8,2019-10,Male,31-50,DTH:PAL,Capital Region
9,2019-10,Male,31-50,DTH:PAL,Capital Region


### Remove dataframe no longer needed

In [18]:
del(demographics_df)

### Group by all needed combinations and count number of occurances

In [19]:
bed_days_df['TotalPatientDays'] = bed_days_df.groupby('Month_Year')['Month_Year'].transform('count')
bed_days_df['TotalPatientDaysBySex'] = bed_days_df.groupby(['Month_Year', 'Sex'])['Month_Year'].transform('count')
bed_days_df['TotalPatientDaysByAge'] = bed_days_df.groupby(['Month_Year', 'Age Group'])['Month_Year'].transform('count')
bed_days_df['BoardPatientDays'] = bed_days_df.groupby(['Month_Year', 'Health Board'])['Month_Year'].transform('count')
bed_days_df['BoardPatientDaysBySex'] = bed_days_df.groupby(['Month_Year', 'Health Board', 'Sex'])['Month_Year'].transform('count')
bed_days_df['BoardPatientDaysByAge'] = bed_days_df.groupby(['Month_Year', 'Health Board', 'Age Group'])['Month_Year'].transform('count')
bed_days_df['UnitPatientDays'] = bed_days_df.groupby(['Month_Year', 'Ward Code'])['Month_Year'].transform('count')
bed_days_df['UnitPatientDaysBySex'] = bed_days_df.groupby(['Month_Year', 'Ward Code', 'Sex'])['Month_Year'].transform('count')
bed_days_df['UnitPatientDaysByAge'] = bed_days_df.groupby(['Month_Year', 'Ward Code', 'Age Group'])['Month_Year'].transform('count')
bed_days_df = bed_days_df.drop_duplicates()
bed_days_df

,Month_Year,Sex,Age Group,Ward Code,Health Board,TotalPatientDays,TotalPatientDaysBySex,TotalPatientDaysByAge,BoardPatientDays,BoardPatientDaysBySex,BoardPatientDaysByAge,UnitPatientDays,UnitPatientDaysBySex,UnitPatientDaysByAge
0,2019-10,Male,31-50,DTH:PAL,Capital Region,1362,667,415,683,335,276,77,57,27
27,2019-11,Male,31-50,DTH:PAL,Capital Region,1276,584,324,476,206,81,60,48,17
31,2019-05,Female,17-30,FTG:PSY,Forest Heights,1402,613,313,486,233,83,79,30,12
43,2019-06,Female,17-30,FTG:PSY,Forest Heights,1223,583,206,348,162,52,75,19,10
53,2019-06,Female,31-50,SSG:ONC,Capital Region,1223,583,335,540,295,191,27,24,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79977,2022-01,Female,51-65,UHC:1,Capital Region,1647,903,264,929,480,180,91,74,24
80059,2020-06,Female,51-65,ORI:GENM,Forest Heights,1452,731,203,393,245,91,66,26,51
80102,2021-09,Female,65+,ORI:1,Forest Heights,1303,610,498,281,146,70,69,56,13
80117,2023-08,Male,65+,DTH:2,Capital Region,952,490,367,434,224,196,46,11,35


### Merge results of counting by groups with patient data

In [20]:
final_df = pd.merge(df,bed_days_df,on=['Month_Year', 'Sex', 'Age Group', 'Ward Code', 'Health Board'],how='left')
final_df.head(5)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection,...,Month_Year,TotalPatientDays,TotalPatientDaysBySex,TotalPatientDaysByAge,BoardPatientDays,BoardPatientDaysBySex,BoardPatientDaysByAge,UnitPatientDays,UnitPatientDaysBySex,UnitPatientDaysByAge
0,154907,49,Male,2019-10-05,2019-11-04,2021-05-31,Staphylococcus aureus,DTH:PAL,0,604,...,2021-05,1270.0,716.0,310.0,683.0,419.0,183.0,32.0,24.0,14.0
1,380679,28,Female,2019-05-20,2019-06-10,2022-07-09,Enterococcus faecalis,FTG:PSY,0,1146,...,2022-07,1316.0,681.0,208.0,375.0,259.0,95.0,11.0,11.0,8.0
2,191421,37,Female,2019-06-30,2019-07-24,2023-09-04,Enterococcus faecalis,SSG:ONC,0,1527,...,2023-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,906309,51,Male,2021-08-17,2021-09-12,2021-08-19,Escherichia coli,UHC:ICUA,1,2,...,2021-08,1422.0,728.0,229.0,676.0,385.0,118.0,26.0,22.0,15.0
4,527023,84,Male,2020-12-12,2021-01-09,2019-08-03,Klebsiella pneumoniae,DTH:1,0,-497,...,2019-08,1188.0,554.0,478.0,603.0,258.0,244.0,32.0,8.0,32.0


### Final step is to create a dummy entry for all combinations of date, sex, ward, hospital etc so that when plotting in Tableau, there are no missing values

In [22]:
unique_units = sorted(final_df['Ward Code'].unique())
unique_dates = sorted(final_df['Month_Year'].unique())
unique_ages = sorted(final_df['Age Group'].unique())
unique_sexes = sorted(final_df['Sex'].unique())
all_combinations = list(product(unique_units, unique_dates, unique_ages, unique_sexes))
all_combinations_df = pd.DataFrame(all_combinations)
all_combinations_df = all_combinations_df.rename(columns={0 : "Ward Code", 1: "Month_Year", 2: "Age Group", 3: "Sex"})
all_combinations_df = pd.merge(all_combinations_df, health_data, on='Ward Code', how='left')
all_combinations_df.head(5)

,Ward Code,Month_Year,Age Group,Sex,Ward Name,Hospital Code,Hospital Name,Acute Status,Health Board Code,Health Board,Population
0,BGH:1,2019-01,17-30,Female,Ward 1,BDH101,Bright Dawn Hospital,Acute,MPK,Middlepark,8000
1,BGH:1,2019-01,17-30,Male,Ward 1,BDH101,Bright Dawn Hospital,Acute,MPK,Middlepark,8000
2,BGH:1,2019-01,31-50,Female,Ward 1,BDH101,Bright Dawn Hospital,Acute,MPK,Middlepark,8000
3,BGH:1,2019-01,31-50,Male,Ward 1,BDH101,Bright Dawn Hospital,Acute,MPK,Middlepark,8000
4,BGH:1,2019-01,51-65,Female,Ward 1,BDH101,Bright Dawn Hospital,Acute,MPK,Middlepark,8000


In [23]:
padding_df = pd.DataFrame(columns=final_df.columns.tolist())
padding_df['Ward Code'] = all_combinations_df['Ward Code']
padding_df['Month_Year'] = all_combinations_df['Month_Year']
padding_df['Age Group'] = all_combinations_df['Age Group']
padding_df['Sex'] = all_combinations_df['Sex']
padding_df['Ward Name'] = all_combinations_df['Ward Name']
padding_df['Hospital Code'] = all_combinations_df['Hospital Code']
padding_df['Hospital Name'] = all_combinations_df['Hospital Name']
padding_df['Acute Status'] = all_combinations_df['Acute Status']
padding_df['Health Board Code'] = all_combinations_df['Health Board Code']
padding_df['Health Board'] = all_combinations_df['Health Board']
padding_df['HAI'] = 0
padding_df.head(10)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection,...,Month_Year,TotalPatientDays,TotalPatientDaysBySex,TotalPatientDaysByAge,BoardPatientDays,BoardPatientDaysBySex,BoardPatientDaysByAge,UnitPatientDays,UnitPatientDaysBySex,UnitPatientDaysByAge
0,NaN,NaN,Female,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Male,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Female,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Male,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Female,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,Male,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Female,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Male,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Female,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,Male,NaN,NaN,NaN,NaN,BGH:1,0,NaN,...,2019-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final_df = pd.concat([final_df, padding_df])

In [22]:
final_df.tail(5)

,Patient ID,Age,Sex,Unit Admission Date,Unit Discharge Date,Specimen Date,Organism,Ward Code,HAI,Days To Infection,...,TotalPatientDays,TotalPatientDaysBySex,TotalPatientDaysByAge,BoardPatientDays,BoardPatientDaysBySex,BoardPatientDaysByAge,UnitPatientDays,UnitPatientDaysBySex,UnitPatientDaysByAge,Health Board
10747,NaN,NaN,Male,NaT,NaT,NaT,NaN,UHC:MAT,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capital Region
10748,NaN,NaN,Female,NaT,NaT,NaT,NaN,UHC:MAT,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capital Region
10749,NaN,NaN,Male,NaT,NaT,NaT,NaN,UHC:MAT,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capital Region
10750,NaN,NaN,Female,NaT,NaT,NaT,NaN,UHC:MAT,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capital Region
10751,NaN,NaN,Male,NaT,NaT,NaT,NaN,UHC:MAT,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Capital Region


In [26]:
final_df.to_csv('data.csv', index=False)